# Логистическая регрессия. Log Loss

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 5, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2,  1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

In [3]:
#global_functions
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

# copy X array
X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

## 1. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log

In [4]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred + 1e-299) + (1.0 - y) * np.log(1.0 - y_pred + 1e-299))
    err = np.sum(err)
    return err

## 2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным

In [5]:
def eval_model(X, y, iterations, alpha=1e-4, output=True):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T))
        if output and i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [6]:
W = eval_model(X_st, y, iterations=1000000, alpha=1e-2)

100000 [-10.74417094  -1.38394305  -2.42070613   9.13379901] 0.23214376711914236
200000 [-15.42675299  -1.7813434   -3.82857757  12.91697903] 0.19343511937930052
300000 [-19.03147665  -2.10012227  -4.88442605  15.84668363] 0.17055165430890828
400000 [-21.9984015   -2.36612288  -5.74167338  18.25876214] 0.15508567715960073
500000 [-24.52626334  -2.59421122  -6.46565423  20.31251846] 0.14388130886831396
600000 [-26.73370635  -2.79394547  -7.0941108   22.10416896] 0.13535239004100927
700000 [-28.69811567  -2.97178775  -7.65111645  23.6967112 ] 0.12860871579517866
800000 [-30.47261246  -3.13227528  -8.15292     25.1334636 ] 0.1231138543076761
900000 [-32.09507997  -3.27869631  -8.6109551   26.44534733] 0.11852642476065993
1000000 [-33.59338441  -3.41349945  -9.03354044  27.65512062] 0.1146193340950584


## 3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba)

In [7]:
def calc_pred_proba (W, X):
    return sigmoid(X.T @ W)

In [8]:
print(calc_pred_proba(W, X_st))

[3.76929812e-01 1.43639783e-02 9.99999995e-01 1.10353190e-07
 9.32591452e-01 6.42813185e-02 1.00000000e+00 2.06620214e-02
 6.05608740e-01 9.99999456e-01]


## 4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred)

In [9]:
def calc_pred(W, X, threshold=0.6):
    y_pred = calc_pred_proba(W, X) > threshold
    y_pred = y_pred.astype(int)
    return y_pred

In [10]:
y_pred = calc_pred(W, X_st, 0.5)
print(f"y = {y.astype(int)}")
print(f"y_pred = {y_pred}")

y = [0 0 1 0 1 0 1 0 1 1]
y_pred = [0 0 1 0 1 0 1 0 1 1]


## 5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score

In [25]:
def counts(y, y_pred):
    [TP, FP, TN, FN] = [0, 0, 0, 0]
    for i in range(0, len(y)):
        TP += y[i] == 1 and y[i] == y_pred[i]
        FP += y[i] == 1 and y[i] != y_pred[i]
        TN += y[i] == 0 and y[i] == y_pred[i]
        FN += y[i] == 0 and y[i] != y_pred[i]
    return ([TP, FP, TN, FN])

In [27]:
[TP, FP, TN, FN] = counts(y, y_pred)

m = [[TP, FP], [FN, TN]]

print(f"TP = {TP}, FP = {FP}, TN = {TN}, FN = {FN}")
print(f"Матрица ошибок: {m}")

TP = 5, FP = 0, TN = 5, FN = 0
Матрица ошибок: [[5, 0], [0, 5]]


In [34]:
fat = TP / (TP + FN)
print(f'Полнота = {fat}')

Полнота = 1.0


In [35]:
accuracy = sum(y_pred == y) / len(y)
print(f'accuracy = {accuracy}')

accuracy = 1.0


In [36]:
precision = TP / (TP + FP)
print(f'Точность = {precision}')

Точность = 1.0


In [37]:
print(f"F1 = {2 * precision * fat / (fat + precision)}")

F1 = 1.0


## 6. Могла ли модель переобучиться? Почему?
Могла и что-то мне подсказывает из-за малого количества данных, потому и модель подогнала данные под тренеровочные. В качестве решения данной проблемы можно попробовать увеличить порог или собрать (сгенерировать) больше данных.

## 7. Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.

In [38]:
def eval_model_l1(X, y, iterations, alpha=1e-4, lambda_=1e-8, output=True):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_/2 * np.sign(W))
        if output and i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [39]:
W = eval_model_l1(X_st, y, iterations=100000, alpha=1e-2, lambda_=1e-8)

10000 [-2.51838361 -0.94526214  0.40028952  3.13247226] 0.37535792280396973
20000 [-4.27621315 -0.98162742 -0.27399325  4.26180446] 0.3266487344469061
30000 [-5.56017653 -1.03173737 -0.73813177  5.15120024] 0.2999315238051791
40000 [-6.58495576 -1.08568888 -1.09078729  5.89792795] 0.2825328837207596
50000 [-7.45444396 -1.13931748 -1.38008326  6.55223088] 0.2698064830906947
60000 [-8.22275567 -1.19149315 -1.62964483  7.14285019] 0.2597559467158225
70000 [-8.92045267 -1.24198196 -1.8522622   7.68707857] 0.25140035462903876
80000 [-9.56601919 -1.29081555 -2.05544442  8.19585752] 0.24420470961661667
90000 [-10.17128133  -1.3380977   -2.24389221   8.67643211] 0.23785247658579034
100000 [-10.74416694  -1.38394272  -2.42070488   9.1337958 ] 0.23214380612198954


In [40]:
def eval_model_l2(X, y, iterations, alpha=1e-4, lambda_=1e-8, output=True):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + lambda_ * W)
        if output and i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [41]:
W = eval_model_l2(X_st, y, iterations=100000, alpha=1e-2, lambda_=1e-8)

10000 [-2.51838261 -0.94526199  0.40028967  3.1324714 ] 0.37535795650085757
20000 [-4.2762097  -0.98162711 -0.27399234  4.26180181] 0.32664881641443755
30000 [-5.56016979 -1.03173685 -0.73812978  5.1511951 ] 0.29993164968496855
40000 [-6.58494496 -1.08568807 -1.09078399  5.8979196 ] 0.2825330535976823
50000 [-7.45442837 -1.13931629 -1.38007848  6.5522187 ] 0.26980669852812267
60000 [-8.22273464 -1.1914915  -1.62963838  7.14283359] 0.2597562098716665
70000 [-8.92042558 -1.24197978 -1.85225392  7.68705702] 0.251400667702575
80000 [-9.56598545 -1.2908128  -2.05543416  8.19583055] 0.24420507458719531
90000 [-10.17124042  -1.33809431  -2.24387982   8.67639926] 0.23785289510470556
100000 [-10.74411836  -1.38393864  -2.42069023   9.13375667] 0.2321442794820287
